In [1]:
import time
import redpitaya_scpi as scpi
import matplotlib.pyplot as plt
import numpy as np
from datetime import date
import struct
from ipywidgets import widgets
import scipy.interpolate as interplt
import os
import keyboard
import tikzplotlib
from IPython.display import HTML
from ipypublish import nb_setup
from scipy import signal

pd = nb_setup.setup_pandas(escape_latex = False)
pi = np.pi

folder = './data/{}/{}_{}/{}/{}h{}/'.format(date.today().strftime('%Y'), date.today().strftime('%m'), date.today().strftime('%B'), date.today().strftime('%d'), str(time.localtime().tm_hour).zfill(2), str(time.localtime().tm_min).zfill(2))

def saveNotebook():
    ### I am very sorry for this, found no other way
    keyboard.press_and_release('ctrl+s')
    time.sleep(3)

def previousPowerOf2(n):
    while (n & n - 1):
        n = n & n - 1   
    return int(n)

def getRFInputs(dt=1.0) :
    prescaler = previousPowerOf2(int(min(10, dt)* 125e6 / 16384))

    rp_s = scpi.scpi('rp-f06549.local')

    rp_s.rst()

    rp_s.tx_txt('ACQ:DATA:FORMAT BIN')
    rp_s.tx_txt('ACQ:DATA:UNITS RAW')

    rp_s.tx_txt('ACQ:SOUR1:GAIN HV')
    rp_s.tx_txt('ACQ:SOUR2:GAIN HV')


    rp_s.tx_txt('ACQ:TRIG:DLY 0')
    rp_s.tx_txt('ACQ:DEC '+str(prescaler))

    rp_s.tx_txt('ACQ:START')

    while 1:
        rp_s.tx_txt('ACQ:TRIG:STAT?')
        if rp_s.rx_txt() == 'TD':
            break
    time.sleep(prescaler/125e6*16384 + 0.5)

    rp_s.tx_txt('ACQ:SOUR1:DATA?')
    buff_byte = rp_s.rx_arb()
    buff = np.array([struct.unpack('!h',bytearray(buff_byte[i:i+2]))[0] for i in range(0, len(buff_byte), 2)]) / 400
    V = buff

    rp_s.tx_txt('ACQ:SOUR2:DATA?')
    buff_byte = rp_s.rx_arb()
    buff = np.array([struct.unpack('!h',bytearray(buff_byte[i:i+2]))[0] for i in range(0, len(buff_byte), 2)]) / 400
    P = buff
    t =  np.arange(0, prescaler/125e6*16384, prescaler/125e6)
    
    return data(t, V), data(t, P)

class parameter(widgets.FloatText):
    def __init__(self, value, minv, maxv, inc=1):
        super().__init__()
        self.value=value
        self.min=minv
        self.max=maxv
        self.step=inc
        self.disabled=False
        self.continuous_update=True
        self.readout=True
        self.readout_format='.1f'
        
class graph:
    def __init__(self, x, y, xlabel='', ylabel='', title='', symbol='-', color='C0'):
        self.x = np.real(x*np.conj(x) / (np.real(x) + 1e-42))
        self.y = np.real(y*np.conj(y) / (np.real(y) + 1e-42))
        self.xlabel = xlabel
        self.ylabel = ylabel
        self.title = title
        self.symbol=symbol
        self.color=color
    
def plotGraph(graph1, graph2 = None, filename = 'untitled', log=False, points = False, save=True):
    global folder

    try:
        os.makedirs(folder[:len(folder)-1])    
    except FileExistsError:
        pass


    with plt.style.context('default'):
        if graph2==None:
            fig, ax1 = plt.subplots(1, 1, figsize=(18, 8))
        else:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

            if points : ax2.plot(graph2[0].x, graph2[0].y, 'x', color='C1')
            ax2.plot(graph2[0].x, graph2[0].y, graph2[0].symbol, color=graph2[0].color)

            ax2.set(xlabel=graph2[0].xlabel, ylabel=graph2[0].ylabel, title=graph2[0].title)

            for g in graph2[1:]:
                ax2.plot(g.x, g.y, g.symbol, color=g.color)
            if log: ax2.set_xscale('log')

        if points : ax1.plot(graph1[0].x, graph1[0].y, 'x', color='C1') 
        ax1.plot(graph1[0].x, graph1[0].y, graph1[0].symbol, color=graph1[0].color)

        for g in graph1[1:]:
            ax1.plot(g.x, g.y, g.symbol, color=g.color)
        ax1.set(xlabel=graph1[0].xlabel, ylabel=graph1[0].ylabel, title=graph1[0].title)
        if log: ax1.set_xscale('log')
        
        if save: 
            tikzplotlib.save(folder + sampleName + filename+'.pgf')  
            plt.savefig(folder + sampleName + filename+'.png')
        plt.show()
    
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx
    
class data:
    def __init__(self, t, data):
        self.t = t
        self.y = data
        self.n = len(t)
        self.maxt = np.max(self.t)
        self.max = np.max(self.y)
        self.freq = np.array([])
        self.ampFT = np.array([])
        self._FT = np.array([])
        self._idx = np.array([])
        self._freqFT = np.array([])

        self.FourierTransform()
        
    def __updateVariables(self):
        self.freq = self._freqFT[self._idx]
        self.ampFT = (self._FT* np.conj(self._FT) / self.n)[self._idx]
        self.ampFT /= np.sum(self.ampFT)

    def copy(self):
        copy = data(self.t, self.y)
        copy.FourierTransform(max(self.freq))
        return copy

    def FourierTransform(self, maxf = None):
        psd = np.fft.fft(self.y, self.n) #computes the fft
        freq = np.arange(self.n) / self.maxt #frequency array 
        
        self._idx  = np.arange(1, np.floor(self.n/2), dtype=np.int32)
        self._freqFT = freq
        self._FT = psd
        maxIdx = self.n
        
        if maxf != None and maxf < max(freq):   
            maxIdx = find_nearest(self._freqFT[self._idx],  maxf)
            self._idx = np.arange(1, np.floor(maxIdx), dtype=np.int32) 
            self._freqFT = freq[:maxIdx]
            self._FT = psd[:maxIdx]
        
        self.__updateVariables()
        
        return self._freqFT, self._FT, self._idx 

    def Denoise(self, q):
        f = interplt.splrep(self.t, self.y,k=5 ,s=q*1e-4)

        output = self.copy()
        output.y = interplt.splev(self.t,f)
        return output

    
    def getFrequency(self):
        return self.freq[np.where(self.ampFT == max(self.ampFT))[0]][0]

    def find_period(self, i, freq=None):
        if freq==None:
            idx = find_nearest(self.t, i/(self.getFrequency()))
        else:
            idx = find_nearest(self.t, i/freq)
        return idx, self.t[idx]
    
def changeParameters(lr = True, nbp = True, mfa = True):

    global sampleName_p
    global measureDuration_p
    global Rload_p
    global nbCycles_p
    global Rcircuit_p
    global fourrierMaxFreq_p

    try:
        parameters = np.loadtxt('./src/parameters.csv', delimiter=",", comments='#', dtype=str)

        sampleName = parameters[0]
        measureDuration = float(parameters[1])
        Rload = parameters[2]
        nbCycles = float(parameters[3])
        Rcircuit = float(parameters[4])
        fourrierMaxFreq = float(parameters[5])

        sampleName_p          = widgets.Text( value=sampleName, placeholder='Custom Name')
        measureDuration_p     = parameter(measureDuration, 0, 30, 0.1)
        Rload_p               = parameter(Rload, 0, 1e9)
        nbCycles_p            = parameter(nbCycles, 0, 1e9)
        Rcircuit_p            = parameter(Rcircuit, 0, 1e9)
        fourrierMaxFreq_p     = parameter(fourrierMaxFreq, 0, 1e9)

    except:
        sampleName_p          = widgets.Text( value='', placeholder='Custom Name')
        measureDuration_p     = parameter(0, 0, 30, 0.1)
        Rload_p               = parameter(0, 0, 1e9)
        nbCycles_p            = parameter(0, 0, 1e9)
        Rcircuit_p            = parameter(0, 0, 1e9)
        fourrierMaxFreq_p     = parameter(0, 0, 1e9)

    display( widgets.HBox([   widgets.Label('Sample name :'), sampleName_p ]))
    display( widgets.HBox([   widgets.Label('Measure duration [s] :'), measureDuration_p ]))
    if lr: display( widgets.HBox([   widgets.Label('Load resistance [Ohms] :'), Rload_p ]))
    if nbp : display( widgets.HBox([    widgets.Label('Nombre of periods [1] :'), nbCycles_p ]))
    display( widgets.HBox([    widgets.Label('Circuit resistance [Ohms] :'), Rcircuit_p ]))
    if mfa : display( widgets.HBox([    widgets.Label('Maximum frequency for analysis: [Hz] :'), fourrierMaxFreq_p ]))